In [1]:
!git clone https://github.com/quicksilverTrx/llm-forge

Cloning into 'llm-forge'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 76 (delta 24), reused 58 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (76/76), 57.92 KiB | 4.46 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [2]:
!ls

llm-forge  sample_data


In [3]:
!pwd

/content


In [4]:
%cd llm-forge

/content/llm-forge


In [5]:
!git checkout GQA_bug_fixes

Branch 'GQA_bug_fixes' set up to track remote branch 'GQA_bug_fixes' from 'origin'.
Switched to a new branch 'GQA_bug_fixes'


In [6]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [7]:
!pwd

/content/llm-forge


In [8]:
%cd src/model

/content/llm-forge/src/model


In [9]:
def report_gpu_usage(stage,step=0):
    allocated = torch.cuda.memory_allocated() / 1024**2  # Convert to MB
    reserved = torch.cuda.memory_reserved() / 1024**2  # Convert to MB
    if step%100 ==0:
      print(f"[{stage}] GPU Memory Allocated: {allocated:.2f} MB")
      print(f"[{stage}] GPU Memory Reserved: {reserved:.2f} MB\n")

In [10]:
from transformers import GPT2Model
import torch

In [11]:
report_gpu_usage("before model load")

[before model load] GPU Memory Allocated: 0.00 MB
[before model load] GPU Memory Reserved: 0.00 MB



In [12]:
!ls

model_patcher.py  train_colab.ipynb  train_local.ipynb	train_trigger.py  train_utils.py


In [13]:
from transformers import GPT2Model, GPT2Tokenizer
import sys
import os
sys.path.append('/content/llm-forge')
import torch
from torch.optim import AdamW
from src.attention.optimizations.grouped_query_attention import GroupedQueryAttention, GQAConfig
from model_patcher import patch_attention_in_gpt2, add_rope_to_attention, remove_position_embeddings
from train_utils import  train_epoch, evaluate, prepare_wikitext2, LanguageModelingWrapper



In [14]:
train_epoch

<function train_utils.train_epoch(model, data_loader, optimizer, device)>

In [15]:
report_gpu_usage("initial")

[initial] GPU Memory Allocated: 0.00 MB
[initial] GPU Memory Reserved: 0.00 MB



In [16]:


model = GPT2Model.from_pretrained('gpt2')
model.to('cuda')  # Load to GPU

# Print model parameter count and estimated size
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {num_params / 1e6:.2f}M")  # Convert to millions
print(f"Model size in memory (FP32): {num_params * 4 / (1024**2):.2f} MB")
print(f"Model size in memory (FP16): {num_params * 2 / (1024**2):.2f} MB")
report_gpu_usage("after model load")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Number of parameters: 124.44M
Model size in memory (FP32): 474.70 MB
Model size in memory (FP16): 237.35 MB
[after model load] GPU Memory Allocated: 487.47 MB
[after model load] GPU Memory Reserved: 542.00 MB



In [17]:
# Delete the model
del model

# Free up unused GPU memory
torch.cuda.empty_cache()

In [18]:
# Load base model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_model = GPT2Model.from_pretrained('gpt2')
gpt2_config = base_model.config
# 1. Patch single attention variant
gqa_config = GQAConfig(
        num_query_heads=gpt2_config.n_head,  # 12 for GPT-2 small
        num_kv_heads=gpt2_config.n_head //3,  # Reduce KV heads by factor of 3
        head_dim=gpt2_config.n_embd // gpt2_config.n_head,  # 768/12 = 64
        dropout=gpt2_config.attn_pdrop
    )

print(gqa_config)

flash_model = patch_attention_in_gpt2(base_model, GroupedQueryAttention,gqa_config)


final_model = flash_model
final_model = LanguageModelingWrapper(final_model)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token

if tokenizer.pad_token_id is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

final_model.to(device)
report_gpu_usage("GQA model load")

GQAConfig(num_query_heads=12, num_kv_heads=4, head_dim=64, dropout=0.1, max_seq_length=8192, use_bias=False)
Using random initialization for layer 0
Using random initialization for layer 1
Using random initialization for layer 2
Using random initialization for layer 3
Using random initialization for layer 4
Using random initialization for layer 5
Using random initialization for layer 6
Using random initialization for layer 7
Using random initialization for layer 8
Using random initialization for layer 9
Using random initialization for layer 10
Using random initialization for layer 11


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

[GQA model load] GPU Memory Allocated: 439.33 MB
[GQA model load] GPU Memory Reserved: 552.00 MB



In [19]:
# Print model parameter count and estimated size
num_params = sum(p.numel() for p in final_model.parameters())
print(f"Number of parameters: {num_params / 1e6:.2f}M")  # Convert to millions
print(f"Model size in memory (FP32): {num_params * 4 / (1024**2):.2f} MB")
print(f"Model size in memory (FP16): {num_params * 2 / (1024**2):.2f} MB")
report_gpu_usage("after model load")

Number of parameters: 114.97M
Model size in memory (FP32): 438.57 MB
Model size in memory (FP16): 219.29 MB
[after model load] GPU Memory Allocated: 439.33 MB
[after model load] GPU Memory Reserved: 552.00 MB



In [20]:
# torch.cuda.empty_cache()

In [21]:
# del final_model

In [22]:
import gc
gc.collect()

573

In [23]:
train_loader, eval_loader = prepare_wikitext2(tokenizer,batch_size=8, seq_length=512)

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/685k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.07M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/618k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [24]:
report_gpu_usage("GQA model load")

[GQA model load] GPU Memory Allocated: 439.33 MB
[GQA model load] GPU Memory Reserved: 552.00 MB



In [25]:
print(final_model)

LanguageModelingWrapper(
  (base_model): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GroupedQueryAttention(
          (q_proj): Linear(in_features=768, out_features=768, bias=False)
          (k_proj): Linear(in_features=768, out_features=256, bias=False)
          (v_proj): Linear(in_features=768, out_features=256, bias=True)
          (out_proj): Linear(in_features=768, out_features=768, bias=False)
          (attention): ScaledDotProductAttention(
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActi

In [26]:
from torch.cuda.amp import GradScaler, autocast
import gc
def train_epoch(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    total_tokens = 0
    scaler = GradScaler()


    for step, batch in enumerate(data_loader):
        batch = {k : v.to(device) for k, v in batch.items()}
        # if(step%50==0):
        #   report_gpu_usage(f"{step} start")
        with autocast():
          outputs = model(input_ids=batch["input_ids"],
              attention_mask=batch["attention_mask"],
              labels=batch["labels"])
          loss = outputs.loss

        batch_tokens = batch["input_ids"].numel()
        total_loss += loss.item() * batch_tokens
        total_tokens += batch_tokens


        # Scale loss for FP16
        scaler.scale(loss).backward()

        # Step the optimizer using scaled gradients
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

        if(step%50==0):
            print(f"step :{step} training loss : {loss.item()}")

        del batch, outputs, loss
        torch.cuda.empty_cache()  # Optionally clear GPU memory
        gc.collect()
        report_gpu_usage(f"{step}",step)

@torch.no_grad()
def evaluate(model, data_loader, device):
    model.eval()
    total_loss = 0
    total_tokens = 0

    for step,batch in enumerate(data_loader):
        batch = {k : v.to(device) for k, v in batch.items()}
        with  autocast():
            outputs = model(input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                labels=batch["labels"])
            loss = outputs.loss

        batch_tokens = batch["input_ids"].numel()
        total_loss += loss.item() * batch_tokens
        total_tokens += batch_tokens
        del batch, outputs, loss
        gc.collect()
        torch.cuda.empty_cache()  # Optionally clear GPU memory



In [27]:
len(train_loader)

4590

In [28]:
len(eval_loader)

470

In [29]:
train_loader.batch_size

8

In [30]:
# Prepare data


optimizers =  AdamW(final_model.parameters(), lr=5e-5)

for epoch in range (100):
    train_epoch(final_model, train_loader, optimizers, device)
    evaluate(final_model, eval_loader, device)
    torch.cuda.empty_cache()

<ipython-input-26-681bb79df0e1>:7: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-26-681bb79df0e1>:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


step :0 training loss : 7.314083576202393
[0] GPU Memory Allocated: 455.58 MB
[0] GPU Memory Reserved: 908.00 MB

step :50 training loss : 5.464051723480225
step :100 training loss : 5.241898059844971
[100] GPU Memory Allocated: 1342.14 MB
[100] GPU Memory Reserved: 2726.00 MB

step :150 training loss : 5.465328216552734
step :200 training loss : 5.162998199462891
[200] GPU Memory Allocated: 1342.14 MB
[200] GPU Memory Reserved: 2726.00 MB

step :250 training loss : 5.41334867477417
step :300 training loss : 3.652477502822876
[300] GPU Memory Allocated: 1342.14 MB
[300] GPU Memory Reserved: 2726.00 MB

step :350 training loss : 5.35506010055542
step :400 training loss : 5.415769577026367
[400] GPU Memory Allocated: 1342.14 MB
[400] GPU Memory Reserved: 2726.00 MB

step :450 training loss : 5.483786106109619
step :500 training loss : 4.769210338592529
[500] GPU Memory Allocated: 1342.14 MB
[500] GPU Memory Reserved: 2726.00 MB

step :550 training loss : 4.690646648406982
step :600 train

<ipython-input-26-681bb79df0e1>:49: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with  autocast():


step :0 training loss : 4.657044410705566
[0] GPU Memory Allocated: 1342.14 MB
[0] GPU Memory Reserved: 2726.00 MB

step :50 training loss : 3.763437271118164
step :100 training loss : 3.944566249847412
[100] GPU Memory Allocated: 1342.14 MB
[100] GPU Memory Reserved: 2726.00 MB

step :150 training loss : 4.7021026611328125
step :200 training loss : 4.579273700714111
[200] GPU Memory Allocated: 1342.14 MB
[200] GPU Memory Reserved: 2726.00 MB

step :250 training loss : 4.4700775146484375
step :300 training loss : 4.039396286010742
[300] GPU Memory Allocated: 1342.14 MB
[300] GPU Memory Reserved: 2726.00 MB

step :350 training loss : 3.95538067817688
step :400 training loss : 3.9712166786193848
[400] GPU Memory Allocated: 1342.14 MB
[400] GPU Memory Reserved: 2726.00 MB

step :450 training loss : 4.28804874420166
step :500 training loss : 4.626556873321533
[500] GPU Memory Allocated: 1342.14 MB
[500] GPU Memory Reserved: 2726.00 MB



KeyboardInterrupt: 

In [ ]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))


In [ ]:
!nvidia-smi

In [ ]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))
